<a href="https://colab.research.google.com/github/mpadgett004/final_bootcamp_proj/blob/main/full_updated_NB_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "32g") \
    .appName('Top1000Final') \
    .getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://finalbootcampproj.s3.amazonaws.com/updated_total_reddit_comments_NB.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("updated_total_reddit_comments_NB.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+--------------------+
|               score|                body|
+--------------------+--------------------+
|             Neutral|When are the depo...|
|            Positive|            Wow 😂😂|
|             Neutral|Everyone from fom...|
|             Neutral|Use Uphold app to...|
|            Positive|200k @ 0.007 been...|
|            Positive|Deflation isn't b...|
|            Positive|Do some of you kn...|
|            Negative|           Fuck yes!|
|             Neutral| DOnt use robinhood.|
|            Positive|If anyone want to...|
|DMEsZGx9yq9e276E9...|                null|
|            Positive|Doge isn't design...|
|             Neutral|       What deposits|
|            Positive|         I'm alright|
|            Positive|Ok Groyper.  Run ...|
|            Positive|Care to explain a...|
|            Negative|Yeah devastating ...|
| Say things which...|                null|
|             Neutral|I bought Dogecoin...|
|             Neutral|Man, who pay

In [4]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

631144
545108
490239


In [5]:
df.show(100, truncate=False)

+------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|score                                                       |body                                                                                                                                                                                                                                                                                                                       |
+------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
df.printSchema()

root
 |-- score: string (nullable = true)
 |-- body: string (nullable = true)



In [7]:
pandas_df = df.toPandas()
pandas_df

,score,body
0,Positive,I'm glad you made some profit! Buy some more D...
1,Positive,Agreed.. also how they think they are taking d...
2,Neutral,🙌
3,Neutral,MIDNIGHT!
4,Positive,"If I bought $1,000 at 0.03 and then resold at ..."
...,...,...
490234,Neutral,Hands up if you were in before Elon🙌🙌🙌
490235,Neutral,I can’t concentrate at work today...
490236,Positive,This didn’t age well... imagine those who put ...
490237,Neutral,9pm we pump bitcoin. I've got £19 standing by....


In [8]:
pandas_df = pandas_df.loc[((pandas_df['score'] == "Positive") | (pandas_df['score'] == "Negative"))] 
pandas_df

,score,body
0,Positive,I'm glad you made some profit! Buy some more D...
1,Positive,Agreed.. also how they think they are taking d...
4,Positive,"If I bought $1,000 at 0.03 and then resold at ..."
5,Positive,Good job holding @ the real men in this thread...
6,Positive,😂
...,...,...
490229,Positive,On the day the floor was supposed to cave the ...
490230,Negative,Ape Bros this is not a drill (hopefully) buckl...
490232,Positive,diamond hands!!!
490233,Positive,Solid floor 0.064. Not moved for a while.


In [9]:
df_spark = spark.createDataFrame(pandas_df)
df_spark.show(600)

+--------+--------------------+
|   score|                body|
+--------+--------------------+
|Positive|I'm glad you made...|
|Positive|Agreed.. also how...|
|Positive|If I bought $1,00...|
|Positive|Good job holding ...|
|Positive|                  😂|
|Positive|Pretty standard p...|
|Negative|So my 3752 dogeco...|
|Positive| This would be huge!|
|Negative|       Cries in 0.06|
|Positive|go and google led...|
|Positive|i need it back to...|
|Negative|Hope people that ...|
|Negative|Ok officer doofy....|
|Negative|When economy reco...|
|Negative|You bought in at ...|
|Positive|I have a dog ther...|
|Positive|It's my personal ...|
|Positive|Well I'll be on t...|
|Positive|I bought in at .0...|
|Negative|You only lose if ...|
|Positive|these noobs don't...|
|Positive|They said bitcoin...|
|Positive|I hope so my frie...|
|Positive|   Definitely third!|
|Negative|16k Doge @ .049 i...|
|Negative|Think it’s hard f...|
|Negative|I know this is go...|
|Positive|Holding DOGE like...|
|Negative

In [10]:
spark_renamed = df_spark.withColumnRenamed("score", "class").withColumnRenamed("body", "text")
spark_renamed.show(600)
spark_renamed.printSchema()

+--------+--------------------+
|   class|                text|
+--------+--------------------+
|Positive|I'm glad you made...|
|Positive|Agreed.. also how...|
|Positive|If I bought $1,00...|
|Positive|Good job holding ...|
|Positive|                  😂|
|Positive|Pretty standard p...|
|Negative|So my 3752 dogeco...|
|Positive| This would be huge!|
|Negative|       Cries in 0.06|
|Positive|go and google led...|
|Positive|i need it back to...|
|Negative|Hope people that ...|
|Negative|Ok officer doofy....|
|Negative|When economy reco...|
|Negative|You bought in at ...|
|Positive|I have a dog ther...|
|Positive|It's my personal ...|
|Positive|Well I'll be on t...|
|Positive|I bought in at .0...|
|Negative|You only lose if ...|
|Positive|these noobs don't...|
|Positive|They said bitcoin...|
|Positive|I hope so my frie...|
|Positive|   Definitely third!|
|Negative|16k Doge @ .049 i...|
|Negative|Think it’s hard f...|
|Negative|I know this is go...|
|Positive|Holding DOGE like...|
|Negative

In [12]:
from pyspark.sql.functions import length
data_df = spark_renamed.withColumn('length', length(spark_renamed['text']))
data_df.show()

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
|Positive|I'm glad you made...|   106|
|Positive|Agreed.. also how...|    86|
|Positive|If I bought $1,00...|   203|
|Positive|Good job holding ...|   144|
|Positive|                  😂|     1|
|Positive|Pretty standard p...|    52|
|Negative|So my 3752 dogeco...|    44|
|Positive| This would be huge!|    19|
|Negative|       Cries in 0.06|    13|
|Positive|go and google led...|   264|
|Positive|i need it back to...|    25|
|Negative|Hope people that ...|   144|
|Negative|Ok officer doofy....|   117|
|Negative|When economy reco...|   119|
|Negative|You bought in at ...|   124|
|Positive|I have a dog ther...|    38|
|Positive|It's my personal ...|   176|
|Positive|Well I'll be on t...|    88|
|Positive|I bought in at .0...|   181|
|Negative|You only lose if ...|   315|
+--------+--------------------+------+
only showing top 20 rows



In [13]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [15]:
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [16]:
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [17]:
 # Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[5765,399...|
|  0.0|(262145,[38308,41...|
|  0.0|(262145,[25015,44...|
|  0.0|(262145,[14273,21...|
|  0.0|(262145,[83805,26...|
|  0.0|(262145,[5829,230...|
|  1.0|(262145,[169292,1...|
|  0.0|(262145,[16452,26...|
|  1.0|(262145,[77835,88...|
|  0.0|(262145,[36987,51...|
|  0.0|(262145,[41809,83...|
|  1.0|(262145,[15682,19...|
|  1.0|(262145,[6872,320...|
|  1.0|(262145,[8538,794...|
|  1.0|(262145,[15682,49...|
|  0.0|(262145,[54556,65...|
|  0.0|(262145,[968,2099...|
|  0.0|(262145,[2408,315...|
|  0.0|(262145,[2306,234...|
|  1.0|(262145,[31986,32...|
+-----+--------------------+
only showing top 20 rows



In [18]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
# predictor = nb.fit(training)

In [19]:
predictor = nb.fit(training)

In [20]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Negative|                    |     1|  1.0|                  []|                  []|      (262144,[],[])|      (262144,[],[])|(262145,[262144],...|[-0.8963051427705...|[0.68216650129845...|       0.0|
|Negative|     Honest-to-go...|   145|  1.0|[, , , , , honest...|[, , , , , honest...|(262144,[18176,52...|(262144,[18176,52...|(262145,[18176,52...|[-1046.5238862517...|[0.97229108574181.

In [21]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.770764
